In [52]:
!pip install ISLP -q
!pip install great_tables
!pip install linearmodels
!pip install stargazer

  Cloning https://github.com/moskalev/texression.git to /private/var/folders/j1/krsnnj4x2p54phvkb1tg2csr0000gn/T/pip-req-build-wtpblcus
  Running command git clone --filter=blob:none --quiet https://github.com/moskalev/texression.git /private/var/folders/j1/krsnnj4x2p54phvkb1tg2csr0000gn/T/pip-req-build-wtpblcus
  Resolved https://github.com/moskalev/texression.git to commit a5fa7be47fb9147a6714e01f71c08585a2ced5bb
ERROR: git+https://github.com/moskalev/texression.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [54]:
!git clone https://github.com/moskalev/texression.git
import sys
sys.path.append('./texression')

Cloning into 'texression'...
remote: Enumerating objects: 61, done.
remote: Total 61 (delta 0), reused 0 (delta 0), pack-reused 61 (from 1)
Receiving objects: 100% (61/61), 59.32 KiB | 2.82 MiB/s, done.
Resolving deltas: 100% (33/33), done.


In [64]:
import numpy as np
import statsmodels.api as sm
import pandas as pd
import networkx as nx
import linearmodels
from great_tables import GT as GT
from matplotlib.pyplot import subplots
from statsmodels.api import OLS
from linearmodels import PanelOLS
from linearmodels import IV2SLS
from linearmodels.panel import compare
from ISLP.models import ModelSpec as MS
from stargazer.stargazer import Stargazer

df = pd.read_stata('./replication_sample.dta')
df_main = df[df['main_sample']==1]

df_main.columns
print(df_main.columns)

Index(['year', 'newhclust', 'HHI_bar', 'HHI_hat', 'r_hat', 'num_words',
       'num_char', 'long_tail', 'branded', 'numind', 'numberofresults',
       'logr_bar', 'logr_hat', 'diffr_hat', 'logcpc', 'logvol', 'lognum',
       'sim', 'cluster', 'constant', 'evermerged', 'timetoswitch',
       'allP_sample', 'main_sample', 'ind_sample', 'own_branded',
       'other_branded', 'negative_HHIchange', 'shiftmerge', 'sapientmerge',
       'merklemerge', 'forsmanmerge'],
      dtype='object')


In [150]:
df_main['timetoswitch'].describe()
#df_main2 = df_main[df_main['main_sample']==1]

count    16807.000000
mean        -0.330220
std          1.039654
min         -2.000000
25%         -1.000000
50%          0.000000
75%          0.000000
max          2.000000
Name: timetoswitch, dtype: float64

In [3]:
df_main = df_main.set_index(["cluster", "year"])

In [4]:
#table4c1
mod_4c1 = PanelOLS.from_formula('logr_hat ~ 1 + HHI_hat', df_main)
res_4c1 = mod_4c1.fit(cov_type='clustered', cluster_entity=True)
print(res_4c1)

#table4c2
mod_4c2 = PanelOLS.from_formula('logr_hat ~ 1 + HHI_hat + EntityEffects', df_main)
res_4c2 = mod_4c2.fit(cov_type='clustered', cluster_entity=True)
print(res_4c2)

#table4c3
mod_4c3 = PanelOLS.from_formula('logr_hat ~ 1 + HHI_hat + numberofresults + EntityEffects + TimeEffects', df_main)
res_4c3 = mod_4c3.fit(cov_type='clustered', cluster_entity=True)
print(res_4c3)

#table4c4
mod_4c4 = PanelOLS.from_formula('logr_hat ~ 1 + HHI_hat + numberofresults + branded + long_tail + EntityEffects + TimeEffects', df_main)
res_4c4 = mod_4c4.fit(cov_type='clustered', cluster_entity=True)
print(res_4c4)


                          PanelOLS Estimation Summary                           
Dep. Variable:               logr_hat   R-squared:                        0.0652
Estimator:                   PanelOLS   R-squared (Between):              0.0162
No. Observations:               52476   R-squared (Within):               0.0731
Date:                Sun, Oct 20 2024   R-squared (Overall):              0.0652
Time:                        20:06:17   Log-likelihood                 -1.08e+05
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      3657.1
Entities:                        8324   P-value                           0.0000
Avg Obs:                       6.3042   Distribution:                 F(1,52474)
Min Obs:                       2.0000                                           
Max Obs:                       81.000   F-statistic (robust):             952.57
                            

In [5]:
#table4c5
mod_4c5 = IV2SLS.from_formula('logr_hat ~ 1 + [HHI_hat ~ sim]', df_main)
res_4c5 = mod_4c5.fit(cov_type='clustered')
print(res_4c5)

                          IV-2SLS Estimation Summary                          
Dep. Variable:               logr_hat   R-squared:                     -0.9415
Estimator:                    IV-2SLS   Adj. R-squared:                -0.9415
No. Observations:               52476   F-statistic:                    17.141
Date:                Sun, Oct 20 2024   P-value (F-stat)                0.0000
Time:                        20:06:18   Distribution:                  chi2(1)
Cov. Estimator:             clustered                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      13.407     0.7232     18.538     0.0000      11.990      14.825
HHI_hat       -10.932     2.6404    -4.1402     0.00

In [6]:
#table4c5 - first stage
mod_4c5 = PanelOLS.from_formula('HHI_hat ~ 1 + sim', df_main)
res_4c5 = mod_4c5.fit(cov_type='clustered', cluster_entity=True)
print(res_4c5)

fittedHHI_4c5 = res_4c5.fitted_values
df_main['HHI_hat_fitted'] = fittedHHI_4c5

#table4c5 - second stage
mod_4c5 = PanelOLS.from_formula('logr_hat ~ 1 + HHI_hat_fitted', df_main)
res_4c5 = mod_4c5.fit(cov_type='clustered', cluster_entity=True)
print(res_4c5)


                          PanelOLS Estimation Summary                           
Dep. Variable:                HHI_hat   R-squared:                        0.0008
Estimator:                   PanelOLS   R-squared (Between):             -0.0172
No. Observations:               52476   R-squared (Within):               0.0026
Date:                Sun, Oct 20 2024   R-squared (Overall):              0.0008
Time:                        20:06:18   Log-likelihood                    3680.5
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      43.369
Entities:                        8324   P-value                           0.0000
Avg Obs:                       6.3042   Distribution:                 F(1,52474)
Min Obs:                       2.0000                                           
Max Obs:                       81.000   F-statistic (robust):             13.213
                            

In [7]:
#table4c6 - first stage
mod_4c6 = PanelOLS.from_formula('HHI_hat ~ 1 + sim + EntityEffects', df_main)
res_4c6 = mod_4c6.fit(cov_type='clustered', cluster_entity=True)
print(res_4c6)

fittedHHI_4c6 = res_4c6.fitted_values
df_main['HHI_hat_fitted'] = fittedHHI_4c6

#table4c6 - second stage
mod_4c6 = PanelOLS.from_formula('logr_hat ~ 1 + HHI_hat_fitted + EntityEffects', df_main)
res_4c6 = mod_4c6.fit(cov_type='clustered', cluster_entity=True)
print(res_4c6)

                          PanelOLS Estimation Summary                           
Dep. Variable:                HHI_hat   R-squared:                        0.0030
Estimator:                   PanelOLS   R-squared (Between):             -0.0185
No. Observations:               52476   R-squared (Within):               0.0030
Date:                Sun, Oct 20 2024   R-squared (Overall):              0.0006
Time:                        20:06:18   Log-likelihood                 2.146e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      130.81
Entities:                        8324   P-value                           0.0000
Avg Obs:                       6.3042   Distribution:                 F(1,44151)
Min Obs:                       2.0000                                           
Max Obs:                       81.000   F-statistic (robust):             147.01
                            

In [8]:
#table4c7 - first stage
mod_4c7 = PanelOLS.from_formula('HHI_hat ~ 1 + sim + numberofresults + EntityEffects + TimeEffects', df_main)
res_4c7 = mod_4c7.fit(cov_type='clustered', cluster_entity=True)
print(res_4c7)

fittedHHI_4c7 = res_4c7.fitted_values
df_main['HHI_hat_fitted'] = fittedHHI_4c7

#table4c7 - second stage
mod_4c7 = PanelOLS.from_formula('logr_hat ~ 1 + HHI_hat_fitted + numberofresults + EntityEffects + TimeEffects', df_main)
res_4c7 = mod_4c7.fit(cov_type='clustered', cluster_entity=True)
print(res_4c7)

                          PanelOLS Estimation Summary                           
Dep. Variable:                HHI_hat   R-squared:                        0.0027
Estimator:                   PanelOLS   R-squared (Between):             -0.0164
No. Observations:               52476   R-squared (Within):               0.0036
Date:                Sun, Oct 20 2024   R-squared (Overall):              0.0017
Time:                        20:06:18   Log-likelihood                 2.164e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      60.297
Entities:                        8324   P-value                           0.0000
Avg Obs:                       6.3042   Distribution:                 F(2,44148)
Min Obs:                       2.0000                                           
Max Obs:                       81.000   F-statistic (robust):             43.035
                            

In [9]:
#table4c8 - first stage
mod_4c8 = PanelOLS.from_formula('HHI_hat ~ 1 + sim + numberofresults + branded + long_tail + EntityEffects + TimeEffects', df_main)
res_4c8 = mod_4c8.fit(cov_type='clustered', cluster_entity=True)
print(res_4c8)

fittedHHI_4c8 = res_4c8.fitted_values
df_main['HHI_hat_fitted'] = fittedHHI_4c8

#table4c8 - second stage
mod_4c8 = PanelOLS.from_formula('logr_hat ~ 1 + HHI_hat_fitted + numberofresults + branded + long_tail + EntityEffects + TimeEffects', df_main)
res_4c8 = mod_4c8.fit(cov_type='clustered', cluster_entity=True)
print(res_4c8)

                          PanelOLS Estimation Summary                           
Dep. Variable:                HHI_hat   R-squared:                        0.0040
Estimator:                   PanelOLS   R-squared (Between):             -0.0283
No. Observations:               52476   R-squared (Within):               0.0049
Date:                Sun, Oct 20 2024   R-squared (Overall):             -0.0045
Time:                        20:06:19   Log-likelihood                 2.167e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      44.273
Entities:                        8324   P-value                           0.0000
Avg Obs:                       6.3042   Distribution:                 F(4,44146)
Min Obs:                       2.0000                                           
Max Obs:                       81.000   F-statistic (robust):             28.274
                            

In [136]:
OLS_stargazer = Stargazer([res_4c1, res_4c2, res_4c3, res_4c4])

covariates = ['HHI_hat', 'numberofresults', 'branded', 'long_tail']
labels = {'HHI_hat': 'HHI',
          'numberofresults': 'Search results (millions)',
          'branded': 'Branded keyword',
          'long_tail': 'Long-tail keyword'} 
          
OLS_stargazer.covariate_order(covariates)
OLS_stargazer.add_line("Cluster Effects", ['', '\checkmark', '\checkmark', '\checkmark'])
OLS_stargazer.add_line("Year Effects", ['', '', '\checkmark', '\checkmark'])
OLS_stargazer.rename_covariates(labels)
OLS_latex_output = OLS_stargazer.render_latex()
print(OLS_latex_output)

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{4}{c}{\textit{Dependent variable: logr_hat}} \
\cr \cline{2-5}
\\[-1.8ex] & (1) & (2) & (3) & (4) \\
\hline \\[-1.8ex]
 HHI & -2.217$^{***}$ & -2.120$^{***}$ & -2.122$^{***}$ & -2.130$^{***}$ \\
& (0.072) & (0.057) & (0.062) & (0.062) \\
 Search results (millions) & & & 0.252$^{***}$ & 0.263$^{***}$ \\
& & & (0.048) & (0.050) \\
 Branded keyword & & & & 0.396$^{***}$ \\
& & & & (0.059) \\
 Long-tail keyword & & & & -0.091$^{**}$ \\
& & & & (0.040) \\
 Cluster Effects & No & Yes & Yes & Yes \\
 Year Effects & No & No & Yes & Yes \\
\hline \\[-1.8ex]
 Observations & 52476 & 52476 & 52476 & 52476 \\
 N. of groups & 8324 & 8324 & 8324 & 8324 \\
 $R^2$ & 0.065 & 0.073 & 0.075 & 0.078 \\
 Residual Std. Error & 0.500 (df=52474) & 0.372 (df=44151) & 0.376 (df=44148) & 0.383 (df=44146) \\
 F Statistic & 3657.142$^{***}$ (df=2; 52474) & 3491.720$^{***}$ (df=8325; 44151) & 

In [140]:
IV_stargazer = Stargazer([res_4c5, res_4c6, res_4c7, res_4c8])

covariates = ['HHI_hat_fitted', 'numberofresults', 'branded', 'long_tail']
labels = {'HHI_hat_fitted': 'HHI',
          'numberofresults': 'Search results (millions)',
          'branded': 'Branded keyword',
          'long_tail': 'Long-tail keyword'} 
          
IV_stargazer.covariate_order(covariates)
IV_stargazer.add_line("Cluster Effects", ['', '\checkmark', '\checkmark', '\checkmark'])
IV_stargazer.add_line("Year Effects", ['', '', '\checkmark', '\checkmark'])
IV_stargazer.rename_covariates(labels)
IV_latex_output = IV_stargazer.render_latex()
print(IV_latex_output)

\begin{table}[!htbp] \centering
\begin{tabular}{@{\extracolsep{5pt}}lcccc}
\\[-1.8ex]\hline
\hline \\[-1.8ex]
& \multicolumn{4}{c}{\textit{Dependent variable: logr_hat}} \
\cr \cline{2-5}
\\[-1.8ex] & (1) & (2) & (3) & (4) \\
\hline \\[-1.8ex]
 HHI & -10.932$^{***}$ & -4.252$^{***}$ & -4.620$^{***}$ & -4.479$^{***}$ \\
& (1.795) & (1.184) & (1.532) & (1.528) \\
 Search results (millions) & & & 0.206$^{***}$ & 0.225$^{***}$ \\
& & & (0.060) & (0.059) \\
 Branded keyword & & & & 0.458$^{***}$ \\
& & & & (0.074) \\
 Long-tail keyword & & & & -0.049$^{}$ \\
& & & & (0.049) \\
 Cluster Effects & No & Yes & Yes & Yes \\
 Year Effects & No & No & Yes & Yes \\
\hline \\[-1.8ex]
 Observations & 52476 & 52476 & 52476 & 52476 \\
 N. of groups & 8324 & 8324 & 8324 & 8324 \\
 $R^2$ & 0.001 & 0.001 & 0.003 & 0.005 \\
 Residual Std. Error & 0.071 (df=52474) & 0.041 (df=44151) & 0.074 (df=44148) & 0.098 (df=44146) \\
 F Statistic & 68.730$^{***}$ (df=2; 52474) & 38.493$^{***}$ (df=8325; 44151) & 63.39